In [ ]:
import os
from solders.keypair import Keypair
from core.rpc import SolanaRPC
from base58 import b58decode 

# Initialize RPC client
rpc = SolanaRPC()

# Create wallet keypair
wallet = Keypair.from_bytes(b58decode(os.getenv("PRIVATE_KEY_BASE58", "")))

# Check balance
balance = rpc.get_balance(public_key=str(wallet.pubkey()))
print(f"Wallet balance: {balance} lamports")


In [ ]:
from core.jito import test_connection

BLOCK_ENGINE_URL = "https://mainnet.block-engine.jito.wtf"
test_connection(BLOCK_ENGINE_URL)



In [ ]:
import requests

def get_raydium_price(timeout=5):
    url = "https://api.raydium.io/pairs"
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()  # Raise an error on bad status codes
    except Exception as e:
        print("Error fetching Raydium data:", e)
        return None

    data = response.json()
    # Adjust the pair name as needed (here we check for WSOL/USDT)
    for pair in data:
        if pair.get("name") == "WSOL/USDT":
            return pair.get("price")
    return None

if __name__ == "__main__":
    price = get_raydium_price()
    if price is not None:
        print(f"The current WSOL/USDT price on Raydium is: {price}")
    else:
        print("WSOL/USDT pair not found or an error occurred.")


In [ ]:
import requests

def get_wsol_usdc_quote(api_key: str, dex_id: str, timeout=5):
    """
    Fetch a WSOL -> USDC price quote via Expand Network's getprice endpoint,
    using the specified dex_id (Raydium = 2700, Orca = 2600).
    Returns the USDC float amount for 1 SOL, or None if an error occurs.
    """
    url = "https://api.expand.network/dex/getprice"

    headers = {
        "x-api-key": api_key
    }

    # WSOL mint: So11111111111111111111111111111111111111112
    # USDC mint: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
    # 1 SOL in lamports = 1e9
    params = {
        "dexId": dex_id,
        "path": "So11111111111111111111111111111111111111112,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v",
        "amountIn": "1000000000"  # 1 SOL in lamports
    }

    try:
        response = requests.get(url, params=params, headers=headers, timeout=timeout)
        response.raise_for_status()
    except Exception as e:
        print(f"[DEX {dex_id}] Error fetching price data:", e)
        return None

    data = response.json()
    if data.get("status") != 200 or "data" not in data:
        print(f"[DEX {dex_id}] Unexpected response:", data)
        return None

    # amountsOut[1] should be the raw USDC amount
    amounts_out = data["data"].get("amountsOut")
    if not amounts_out or len(amounts_out) < 2:
        print(f"[DEX {dex_id}] Missing amountsOut data:", data)
        return None

    usdc_raw = float(amounts_out[1])
    # USDC has 6 decimals on Solana
    usdc_human = usdc_raw / 1e6
    return usdc_human

if __name__ == "__main__":
    # Replace with your actual Expand Network API key
    API_KEY = "BZfVPGatqf9EbcZHU3ptJ6CJI16Al4tE3RdbsZl8"

    # Raydium = 2700
    raydium_quote = get_wsol_usdc_quote(API_KEY, "2700")
    # Orca = 2600
    orca_quote = get_wsol_usdc_quote(API_KEY, "2600")

    if raydium_quote is not None:
        print(f"1 WSOL is approximately {raydium_quote:.6f} USDC on Raydium.")
    else:
        print("Failed to retrieve Raydium quote.")

    if orca_quote is not None:
        print(f"1 WSOL is approximately {orca_quote:.6f} USDC on Orca.")
    else:
        print("Failed to retrieve Orca quote.")


Error fetching price data: 429 Client Error:  for url: https://api.expand.network/dex/getprice?dexId=2700&path=So11111111111111111111111111111111111111112%2CEPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v&amountIn=1000000000
Failed to retrieve the quote.
